In [25]:
import requests
import pandas as pd
import numpy as np
import json
import folium
from fpdf import FPDF
from selenium import webdriver

api_key = "tL6lWEb1UKgKz1n2FmKGhXFheZNPQdaW"

In [26]:
def initialise_map(start_point, end_point, api_key=api_key, zoom=14, style = "main"):
    maps_url = "http://a.api.tomtom.com/map/1/tile/basic/"+style+"/{z}/{x}/{y}.png?tileSize=512&key="
    
    start = [float(x) for x in start_point.split(',')]
    end = [float(x) for x in end_point.split(',')]
    
    center = [(start[0] + end[0])/2, (start[1] + end[1])/2]
    
    TomTom_map = folium.Map(
    location = center,
    zoom_start = zoom,
    tiles = str(maps_url + api_key),
    attr = 'TomTom')
    
    return TomTom_map

In [27]:
def get_depature_and_destination_coords(start_address, end_address):
    start_address_list = start_address.split()
    end_address_list = end_address.split()

    geolocation_endpoint = f'https://api.tomtom.com/search/2/structuredGeocode.json?&key={api_key}'

    params1 = {
        "countryCode": "DK",
        "streetName": start_address_list[0],
        "streetNumber": start_address_list[1],
        "municipality": start_address_list[2],
        "postalCode": start_address_list[3],
        "limit": "1"
        }
    
    params2 = {
        "countryCode": "DK",
        "streetName": end_address_list[0],
        "streetNumber": end_address_list[1],
        "municipality": end_address_list[2],
        "postalCode": end_address_list[3],
        "limit": "1"
        }

    response1 = requests.get(geolocation_endpoint, params = params1)
    start_address_data = response1.json()
    
    response2 = requests.get(geolocation_endpoint, params = params2)
    end_address_data = response2.json()
    

    start_entry_point = start_address_data["results"][0]["entryPoints"][0]["position"]
    
    end_entry_point = end_address_data["results"][0]["entryPoints"][0]["position"]
    
    
    start_point_string = str(start_entry_point['lat']) + "," + str(start_entry_point['lon'])
    end_point_string = str(end_entry_point['lat']) + "," + str(end_entry_point['lon'])
    
    
    return start_point_string, end_point_string;

In [28]:
def get_fastest_route(start_coords, end_coords ):
    
    route_endpoint = f'https://api.tomtom.com/routing/1/calculateRoute/{start_coords}:{end_coords}/json?&key={api_key}'

    params = {
        "traffic": "false",
        "travelMode": "car",
        "routeType": "fastest",
        "instructionsType": "text",
        "language": "en-US",
        "computeBestOrder": "false",
        }

    response = requests.get(route_endpoint, params=params)
    data = response.json()

    route_geo = data["routes"][0]["legs"][0]["points"]

    points = [(c['latitude'], c['longitude']) for c in route_geo]

    folium.PolyLine(locations=points, color='red').add_to(TomTom_map)
    
    bounds = TomTom_map.get_bounds()
    TomTom_map.fit_bounds(bounds)
    
    instructions_list = data["routes"][0]["guidance"]["instructions"]
    
    instruction_txt = []
    
    for instruct in instructions_list:
        if "combinedMessage" in instruct:
            instruction_txt.append(instruct["combinedMessage"] + ".\n")
        else:   
            instruction_txt.append(instruct["message"] + "\n")
    
    return instruction_txt

In [29]:
start_location = input('Whats your current location? The format is streetname number city postalcode')
end_location = input('Where would you like to go? The format is streetname number city postalcode')

start, end = get_depature_and_destination_coords(start_location, end_location)

TomTom_map = initialise_map(start, end)

instruction_txt = get_fastest_route(start, end)

TomTom_map.save('map.html')

browser = webdriver.Chrome()
browser.get('file:///Users/robertpallesen/Documents/Softwareudvikler/Diskret Matematik/SU_Project/dma_folder/map.html')
browser.save_screenshot('map.png')
browser.quit()


pdf = FPDF()
pdf.add_page()
pdf.image('map.png', x=10, y=10, w=190, h=120)
pdf.ln(125)
pdf.set_font("Arial", size = 8)
for elem in instruction_txt:
    pdf.cell(170, 7, ln = 1, txt = elem)

pdf.output("rutevejledning.pdf")

''